## 卷积神经网络

在之前的神经网络学习过程中，使用的都是全连接神经网络，全连接神经网络对识别和预测都有非常好的效果。在之前使用 MNIST 数据集的实践过程中，输入神经网络的是是一幅 28 行 28 列的 784 个像素点的灰度值，但是仅两层神经网络就有十多万个待训练参数（第一层$784\times128个\omega+128个b$，第二层$128\times10个\omega+10个b$，共 101770 个参数）。

在实际项目中，输入神经网络的是具有更高分辨率的彩色图片，使得送入全连接网络的输入特征数特别多，随着隐藏层数的增加，网络规模过大，待优化参数过多，很容易造成过拟合。**为了减少待训练参数，在实际应用时会先对原始图片进行特征提取，再把提取到的特征送给全连接网络**。而卷积计算就是一种有效的提取图像特征的方法。

### 卷积

卷积（Convolutional）的计算过程：

(1)一般会用一个正方形的卷积核，按指定步长，在输入特征图上滑动，遍历输入特征图中的每个像素点。滑动过程如下所示：

![](http://img.zishu010.com/ArticleImage/2017/8/d658466e93f4488bbdfd0856bceb7b36.gif)

(2)每移动一个步长，卷积核会与输入特征图出现重合区域，重合区域对应元素相乘、求和再加上偏置项得到输出特征的每一个像素点。如果输入特征是单通道灰度图，那么使用的就是深度为 1 的单通道卷积核。那么计算过程如下：

![](https://img-blog.csdnimg.cn/20191106143628622.JPG?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NTY4MDAwNw==,size_16,color_FFFFFF,t_70)

**单通道与卷积核深度为 1 有什么关系呢**？首先要知道图片的色彩是由 RGB 三原色组成的，也就是说，除去图片的长度和宽度，还有一个表示图片色彩通道（channel）数的“深度”。因此在卷积过程中，一定要保持卷积核的通道数与输入特征图的通道数一致，即**输入特征图的深度决定了当前层卷积核的深度**。那么对于单通道的特征图而言，通道数为 1 即深度为 1，自然卷积核的深度也就是 1。

综上所述，如果输入特征是三通道彩色图，则需要使用一个 $3\times3\times3$ 的卷积核，结合上面两幅图片，去理解下面的卷积计算过程：

![](https://img-blog.csdn.net/20160707204048899)

在上面的动图中，使用了两个卷积核 fileter w0 和 filter w1，分别得到了最右边 output 下的两个输出特征图，所以当前层使用了几个卷积核，就有几个输出特征图，即**当前卷积核的个数决定了当前层输出特征图的深度**。如果觉得某层模型的特征提取能力不足，可以在这一层多用几个卷积核提高这一层的特征提取能力。

### 感受野

感受野（Receptive Field）：卷积神经网络各输出特征图中的每个像素点，在原始输入图片上映射区域的大小。简单点说就是，特征图上的一个点对应输入图上的区域。